In [2]:
from scipy.stats import ttest_1samp
import numpy as np

import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

from scipy.stats import norm, t, kstest, shapiro
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
# Need conda install -c conda-forge openpyxl
data = pd.read_csv ('cookie_cats.csv')
data.head(20)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
5,540,gate_40,187,True,True
6,1066,gate_30,0,False,False
7,1444,gate_40,2,False,False
8,1574,gate_40,108,True,True
9,1587,gate_40,153,True,False


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [54]:
data.shape

(90189, 5)

In [7]:
data.describe()

,userid,sum_gamerounds
count,9.018900e+04,90189.000000
mean,4.998412e+06,51.872457
std,2.883286e+06,195.050858
min,1.160000e+02,0.000000
25%,2.512230e+06,5.000000
50%,4.995815e+06,16.000000
75%,7.496452e+06,51.000000
max,9.999861e+06,49854.000000


In [10]:
data.version.value_counts(ascending=True)

gate_30    44700
gate_40    45489
Name: version, dtype: int64

In [12]:
data.retention_1.value_counts(ascending=True)

True     40153
False    50036
Name: retention_1, dtype: int64

In [13]:
data.retention_7.value_counts(ascending=True)

True     16781
False    73408
Name: retention_7, dtype: int64

In [7]:
# data[data['converted']>0].describe()

,id,converted
count,35237.000000,35237.0
mean,788394.376962,1.0
std,91398.565565,0.0
min,630001.000000,1.0
25%,709555.000000,1.0
50%,787633.000000,1.0
75%,867831.000000,1.0
max,945991.000000,1.0


In [14]:
data.isna().sum()

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

In [59]:
k1 = data[data['version']=='gate_30']['retention_1'].sum()
k2 = data[data['version']=='gate_40']['retention_1'].sum()
k1_7 = data[data['version']=='gate_30']['retention_7'].sum()
k2_7 = data[data['version']=='gate_40']['retention_7'].sum()
print(f'{k1},{k2}')
print(f'{k1_7},{k2_7}')

20034,20119
8502,8279


In [15]:
n1 = data[data['version']=='gate_30'].shape[0]
n2 = data[data['version']=='gate_40'].shape[0]
print(f'{n1},{n2}')

44700,45489


In [17]:
print(k1/n1)

0.4481879194630872


In [18]:
print(k2/n2)

0.44228274967574577


In [79]:
from statsmodels.stats import proportion
retention1 = np.array([k1, k2])
overall_gate = np.array([n1, n2])
z_score, z_pvalue = proportion.proportions_ztest(count=retention1, nobs=overall_gate,  alternative='two-sided')
print(f' Z score_1: {z_score:.3f}, P-value_1: {z_pvalue:.3f}')
z_score, z_pvalue = proportion.proportions_ztest(np.array([k1_7, k2_7]), 
                                                   np.array([n1, n2]))
print(f' Z score_7: {z_score:.3f}, P-value_7: {z_pvalue:.3f}')

 Z score_1: 1.784, P-value_1: 0.074
 Z score_7: 3.164, P-value_7: 0.002


In [77]:
def ZandPcount(control,variant):
    z_score, z_pvalue = proportion.proportions_ztest(np.array([control, variant]), np.array([n1, n2]))
    return f'{z_score:.3f}, {z_pvalue:.3f}'
print(f'Z-score and p-value 1-day: {ZandPcount(k1,k2)}\nZ-score and p-value 7-day: {ZandPcount(k1_7,k2_7)}')

Z-score and p-value 1-day: 1.784, 0.074
Z-score and p-value 7-day: 3.164, 0.002


In [28]:
chisq, pvalue, table = proportion.proportions_chisquare(np.array([k1, k2]), 
                                                   np.array([n1, n2]))

print(f'ChiSq: {chisq:.3f}, P-value: {pvalue:.3f}')

ChiSq: 3.183, P-value: 0.074


<b>Проверка мощности

In [43]:
import math
import statsmodels.stats.power as smp
from statsmodels.stats.power import TTestIndPower
from tqdm.notebook import tqdm
from scipy import optimize


plt.style.use('ggplot')

#### Критерий пропорций (нужен для кликов, конверсий)

In [55]:
alpha = 0.05
power = 0.95
n = data.shape[0]
p_x = k1/n1
p_y = k2/n2

h = 2*math.asin(np.sqrt(p_x)) - 2*math.asin(np.sqrt(p_y))
# h - величина эффекта
h

0.011881898259223478

#### расчет мощности

In [56]:
power = smp.zt_ind_solve_power(effect_size=h, nobs1=n, alpha=alpha, alternative='two-sided')
power

0.7133585594607001

#### Расчет количества наблюдений  необходимо для заданного эффекта при с alpha = 5% и power = 95%

In [58]:
number_to_observe = smp.zt_ind_solve_power(effect_size = h, alpha = alpha, power = power, alternative='two-sided')
print(f'Number to observe: {number_to_observe:.0f}')

Number to observe: 90189


In [57]:
# effects = []
sample_size = data.shape[0]
power_analysis = TTestIndPower()
def analysis(effect_size):
    return power_analysis.solve_power(effect_size=effect_size, power=power, alpha = alpha) - sample_size
    # for i in tqdm(range(10,1000)):
    # return effects.append(smp.tt_ind_solve_power(effect_size=None,nobs1 = i, alpha = alpha, power = power))
    # sample_sizes.append(i)
print('Maximum detectable effect size: {0:.2f}'.format(optimize.root_scalar(analysis, bracket=[0.01, 1.0]).root))

Maximum detectable effect size: 0.01


<b>Различия между группами не достоверны по показателю retention 1 day, достоверны по retention 7 day (однако проверка 2-сторонняя, поэтому различия могут быть со знаком минус). То есть различия есть, но в отрицательную сторону, нулевая гипотеза выглядит предпочтительнее в данном случае. 
<p><b>Вывод: изменения точки входа в игру после теста не принимаем